## Jpype

In [ ]:
%load_ext autoreload
%autoreload 2
import nima_io
import nima_io.read as ir

# import skimage.io
# import matplotlib
# import matplotlib.pyplot as plt
# %matplotlib inline
# nd2 = "/home/dan/pims/pims/tests/data/bioformats/cluster.nd2"
import numpy as np
from pathlib import Path

tdata = Path("../../tests/data/")

lif = tdata / "2015Aug28_TransHXB2_50min+DMSO.lif"
img_tile = tdata / "t4_1.tif"  # C=3 T=4 S=15
img_void_tile = tdata / "tile6_1.tif"  # C=4 T=3 S=14 scattered
# imgsingle = tdata / "exp2_2.tif"  # C=2 T=81

# mcts = tdata / "multi-channel-time-series.ome.tif"  # C=3 T=7
# bigtiff = tdata / "LC26GFP_1.tf8"  # bigtiff

slif = str(lif)
simg_tile = str(img_tile)
simg_void_tile = str(img_void_tile)
# simgsingle = str(imgsingle)
# smcts = str(mcts)
# sbigtiff = str(bigtiff)

Core metadata list at:
https://docs.openmicroscopy.org/bio-formats/5.9.1/developers/file-reader.html



In [ ]:
rj = ir.read_jpype(slif)

In [ ]:
rj[0]

In [ ]:
filepath = simg_void_tile
loci = jpype.JPackage("loci")
rdr = loci.formats.ImageReader()
rdr.setMetadataStore(loci.formats.MetadataTools.createOMEXMLMetadata())
rdr.setId(filepath)
xml_md = rdr.getMetadataStore()

In [ ]:
md, mdd = ir.get_md_dict(xml_md, debug=True)
md

In [ ]:
print(md["ImageCount"][0][1], md["PlaneCount"])
print(md["PlanePositionX"][3][1][0])
print(md["ImageAcquisitionDate"])
print(md["PixelsPhysicalSizeX"][0][1][0])
print(md["PixelsType"][0][1])

print(md["PlanePositionZ"])

In [ ]:
md0, rdrdt = ir.read_jpype(simg_tile)
rdrdt, md0

In [ ]:
mmd = rdrdt[2]

In [ ]:
mmd["ArcType"]

In [ ]:
rdr = rdrdt[0]

In [ ]:
rdr.getPixelType(), rdr.getDimensionOrder(),

In [ ]:
r = rdr.getMetadataStoreRoot()

In [ ]:
reader = rdr.getReader()

In [ ]:
r.getImage

In [ ]:
rdr.getIndex(0, 3, 2)

In [ ]:
import jpype

loci = jpype.JPackage("loci")

cs = loci.formats.ChannelSeparator(reader)
fs = loci.formats.Memoizer(reader)